# 1. Exploratory Data Analysis

In [1]:
import pandas as pd
import urllib
from contextlib import closing
import shutil
import os
import re
import numpy as np

# 1.1 The Data
Since 1973, the U.S. Centers for Disease Control and Prevention (CDC) have conducted the [National Survey of Family Growth (NSFG)](https://www.cdc.gov/nchs/nsfg/index.htm). According to their website, the survey:
>"gathers information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men’s and women’s health. The survey results are used by the U.S. Department of Health and Human Services and others to plan health services and health education programs, and to do statistical studies of families, fertility, and health." 

Interesting stuff. We're going to use this data to investigate several questions, such as whether first babies tend to be born late. But before we dive into the data, we need to familiarize ourselves with the study's design.

## NSFG Design & Research Terminology
The NSFG has been conducted seven times; each deployment is called a cycle. We're going to use data from Cycle 6, which was conducted from January 2002 to March 2003.

### Cross-Sectional vs. Longitudinal Studies
The NSFG is a [**cross-sectional**](https://en.wikipedia.org/wiki/Cross-sectional_study) study (also known as a cross-sectional analysis, transverse study, prevalence study). A cross-sectional study is a type of observational study that analyzes data from a population, or a representative subset, at a specific point in time. 

An alternative to cross-sectional studies is the [**longitudinal**](https://en.wikipedia.org/wiki/Longitudinal_study) study (also known as a panel study), which involves repeated observations of the same variables across the same groups (e.g., people) over short or long periods of time. The key difference between a longitudinal and cross-sectional study is that the former makes a series of observations more than once on members of the study population over a period of time.

### Populations & Samples
The goal of a survey is to draw conclusions about a **population**. The NSFG's population of interest is Americans aged 15-44. In an ideal world, the NSFG would survey every single American within that age range. But that isn't feasible, so they instead collect data from a subset of that population called a **sample**. The people who participate in a survey are called **respondents**.

### Representation
In general, samples are designed to be **representative**, which means that every member of the target population has an equal chance of participating. That ideal is hard to achieve in practice, but researchers strive towards it so that they can generalize their studies' conclusions. For example, conclusions drawn from a sample that resulted in only female respondents between the ages of 20 and 30 might not generalize well to the target population of all Americans aged 15-44.

Sometimes, however, researches do not want reprensentative samples. Instead, they'll deliberately **oversample** from certain minority groups within the target population. The NSFG recruited three groups - Hispanics, African-Americans and teenagers - at rates higher than their representation in the U.S. population in order to make sure that the number of respondents in each of these groups was large enough to draw valid statistical inferences. The drawback of oversampling is that it is not as easy to draw conclusions about the target population based on statistics from the oversampled survey. This is a point we'll touch on later.

### Codebooks
Whenever you're working with somebody else's data, you ought to familiarize yourself with the researchers' **codebook**, which documents the design of the study, the survey questions, and the encoding of the responses. The codebook and user's guide for the NSFG data are available [here](http://www.cdc.gov/nchs/nsfg/nsfg_cycle6.htm).

# 1.2 Importing the Data
The data can be accessed via a file transfer protocol and comes as a data file in plain text (ASCII), with fixed width columns. Each line in the file is a record that contains data about one pregnancy. The file name is `2002FemPreg.dat`

The format of the data file is documented in `2002FemPreg.dct`, which is a Stata dictionary file. Stata is a statistical software system; a "dictionary" in this context is a list of variable names, types, and indices that identify where in each line to find each variable.

The function defined below will use `urllib` to download the files and place them within a directory called `nsfg_data` within your current working directory. The function will then parse `2002FemPreg.dat` based on the information in `2002FemPreg.dct` and return a pandas `DataFrame` containing just 9 of the 243 columns.

In [54]:
def get_data():
    columns = {'caseid':"int",'prglngth':"int",'outcome':"int",
               'pregordr':"int",'birthord':"int",'birthwgt_lb':"int",
               'birthwgt_oz':"int",'agepreg':"int",'finalwgt':"float64"}
    
    def get_files(ftp_url):
        out_path = os.path.join(os.getcwd(),"nsfg_data")
        if not os.path.exists(out_path):
            os.makedirs(out_path)

        file_name = os.path.basename(ftp_url)
        with closing(urllib.request.urlopen(ftp_url)) as r:
            file = os.path.join(out_path, file_name)
            with open(file, 'wb') as f:
                shutil.copyfileobj(r, f)

        with open(file, 'r') as f:
            lines = f.readlines()
        return lines
    
    data_url = r'ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NSFG/2002FemPreg.dat'
    data_dict_url = r"ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NSFG/stata/2002FemPreg.dct"
    
    data_lines = get_files(data_url)
    data_dict_lines = get_files(data_dict_url)
    
    def parse_data_dict(lines):
        data_columns = {k:[] for k in ['col_width','col_name']}
        for i, line in enumerate(data_dict_lines):
            try:
                data_columns['col_width'].append(re.search(r"[\d]+",line.split()[0]).group())
                data_columns['col_name'].append(line.split()[2])
            except (IndexError, AttributeError) as e:
                if i == 0 or i == len(data_dict_lines) - 1:
                    pass
                else:
                    print(i)
        return data_columns
    
    data_columns = parse_data_dict(data_dict_lines)
    
    def data_to_df(data_columns, data_lines):
        data = {k:[] for k in data_columns['col_name']}
        for line in data_lines:
            widths = data_columns['col_width']
            names = data_columns['col_name']
            counter = 0
            for i,col_name in zip(widths,names):
                start_index = int(i)-1
                try:
                    end_index = int(widths[counter+1])-1
                    counter += 1
                    data[col_name].append(line[start_index:end_index])
                except IndexError:
                    data[col_name].append(line[start_index:])        
        df = pd.DataFrame(data)[list(columns.keys())]
        return df
    
    df = data_to_df(data_columns, data_lines)
    return df
    

In [55]:
df = get_data()

In [56]:
#view top 5 rows
df.head()

,caseid,prglngth,outcome,pregordr,birthord,birthwgt_lb,birthwgt_oz,agepreg,finalwgt
0,1,39,1,1,1,8,13,3316,6448.271111704751
1,1,39,1,2,2,7,14,3925,6448.271111704751
2,2,39,1,1,1,9,2,1433,12999.542264385902
3,2,39,1,2,2,7,0,1783,12999.542264385902
4,2,39,1,3,3,6,3,1833,12999.542264385902


# 1.3 The Variables
There were 243 variables in total. But we're going to focus on the following 9 variables:

 - `caseid` is the integer ID of the respondent.
 - `prglngth` is the integer duration of the pregnancy in weeks.
 - `outcome` is an integer code for the outcome of the pregnancy. The code $1$ indicates a live birth.
 - `pregordr` is a pregnancy serial number; for example, the code for a respondent's first pregnancy is $1$, for the second pregnancy is $2$, and so on.
 - `birthord` is a serial number for live births; the code for a respondent's first child is $1$, and so on. For outcomes other than live birth, this field is blank.
 - `birthwgt_lb` and `birthwgt_oz` contain the pounds and ounces parts of the birth weight of the baby.
 - `agepreg` is the mother's age at the end of the pregnancy.
 - `finalwgt` is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.
 

# 1.4 Data Cleaning
When you import data like this, you often have to check for errors, deal with special values, convert data into different formats, and perform calculations. These operations are called **data cleaning**. 

The first thing we're going to do is replace missing values (i.e. empty strings `' '`) with `np.nan`.

In [63]:
df.replace(r'^\s+$',np.nan,regex=True,inplace=True)

Then we're going to convert the columns to numeric.

In [64]:
df = df.astype(dtype="float64")

Now we need to treat some individual columns. We'll do this with a function.

`agepreg` contains the mother's age at the end of the pregnancy. In the data, `agepreg` is encoded as an integer number of centiyears. So the first line divides each element of `agepreg` by $100$, yielding a 
floating-point value in years.

`birthwgt_lb` and `birthwgt_oz` contain the weight of the baby, in pounds and ounces, for pregnancies that end in live birth. In addition it uses a few special codes:
 - `97 NOT ASCERTAINED`
 - `98 REFUSED`
 - `99 DONT KNOW`
 
We'll recod those special values to `np.nan` since all mathematical operations return `nan` if either argument is `nan`.

The last line of CleanFemPreg creates a new column `totalwgt_lb` that combines pounds and ounces into a single quantity, in pounds.

In [65]:
def CleanFemPreg(df):
    df.agepreg /= 100.0
    na_vals = [97, 98, 99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)
    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0

In [66]:
CleanFemPreg(df)

# 1.5 Data Validation
When you export data from one environment to another, errors might accidentally be introduced. This means it's a good idea to **validate** your data before diving into analyses.

One way to validate your data is to compute basic statistics and then compare them with the published results (if they exist). For example, the NSFG codebook includes tables that summarize each variable. Here is the table for `outcome`, which encodes the outcome of each pregnancy:

| value | label | Total |
|-------|-------|-------|
| 1 | LIVE BIRTH | 9148 |
| 2 | INDUCED ABORTION | 1862 |
| 3 | STILLBIRTH | 120 |
| 4 | MISCARRIAGE | 1921 |
| 5 | ECTOPIC PREGNANCY | 190 |
| 6 | CURRENT PREGNANCY | 352 |

We can reproduce this table ourselves using the `value_counts()` method on the pandas Series:

In [69]:
df.outcome.value_counts(sort=False)

1.0    9148
2.0    1862
4.0    1921
5.0     190
3.0     120
6.0     352
Name: outcome, dtype: int64

If you comparing the results of `value_counts()` with the published table, it looks like the values match up. We can do the same thign with the published table for `birthwgt_lb`:

| value | label | Total |
|-------|-------|-------|
| . | INAPPLICABLE | 4449 |
| 0-5 | UNDER 6 POUNDS | 1125 |
| 6 | 6 POUNDS | 2223 |
| 7 | 7 POUNDS | 3049 |
| 8 | 8 POUNDS | 1889 |
| 9-95 | 9 POUNDS OR MORE | 799 |

In [70]:
df.birthwgt_lb.value_counts(sort=False)

8.0     1889
7.0     3049
6.0     2223
4.0      229
5.0      697
10.0     132
12.0      10
14.0       3
3.0       98
1.0       40
2.0       53
0.0        8
51.0       1
9.0      623
11.0      26
13.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

The counts for 6, 7, and 8 pounds check out, and if you add up the counts for 0-5 and 9-95, they check out, too. But if you look more closely, you will notice one value that has to be an error:  a 51 pound baby!

To deal with that error, we'll recode it to `np.nan` using `.loc`, which provides a lot of ways to select rows and columns from a DataFrame. In this example, the first expression in brackets is the row indexer, which is a boolean, and the the second expression indicates the column name. After using `.loc` to slice the dataframe, we use the assignment operator to set the value(s) there to `np.nan`.

In [73]:
df.loc[df.birthwgt_lb > 20, 'birthwgt_lb'] = np.nan

# 1.6 Data Interpretation
To work with data effectively, you have to simultaneously think both statstically and contextually. As an example with our data, thinking statisically might mean a basic understanding of the data's structure and encodings. Thinking contextually would mean remembering that this dataset describes a very intimate and emotiona facet of human life:  pregnancy. As we'll see below, these two strands of thinking can be woven together to tell a very compelling story.

In [92]:
for i, values in enumerate(df[df['caseid'] == 10229][['prglngth','outcome']].values):
    if values[1] == 4:
        outcome = "miscarriage"
    else:
        outcome = "live birth"
    print("For caseid #{}, pregnancy #{} lasted {} weeks, ending in a {}".format(10229,i,values[0],outcome))

For caseid #10229, pregnancy #0 lasted 2.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #1 lasted 3.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #2 lasted 4.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #3 lasted 2.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #4 lasted 3.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #5 lasted 13.0 weeks, ending in a miscarriage
For caseid #10229, pregnancy #6 lasted 43.0 weeks, ending in a live birth


Above, we printed the `prglngth` and `outcome` for one of the survey's respondents: `caseid` 10229. This small slice of the data tells the story of a woman who was pregnant six times, each time ending in a miscarriage, before having a pregnancy that ended in a live birth after 43 weeks. This is a moving story and should immediately remind us of our duty to faithfully process and represent our data.

# 1.7 Writing Data
There are [many ways](https://pandas.pydata.org/pandas-docs/stable/api.html#id12) to serialize/convert a pandas `DataFrame` so that you can store it locally. We'll simply write our data to a `.csv` file so that we can quickly and easily access it later.

In [94]:
df.to_csv(r'nsfg_data.csv',index=False)

# 1.8 Exercises

#### Exercise 1.8.1
> Create a function that replicates the print statements for caseid 10229 above. Make the function accept a caseid as a parameter. Then use the function on a few different caseids to see what kind of story gets told.

In [ ]:
# your code here


#### Exercise 1.8.2
> The variable `pregnum` is a recode that indicates how many times each respondent has been pregnant. Print the `value_counts` for this variable and compare them to the published results in the NSFG [codebook](ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NSFG/Cycle6Codebook-Pregnancy.pdf).

In [93]:
# your code here


,caseid,prglngth,outcome,pregordr,birthord,birthwgt_lb,birthwgt_oz,agepreg,finalwgt,totalwgt_lb
0,1.0,39.0,1.0,1.0,1.0,8.0,13.0,0.3316,6448.271112,8.8125
1,1.0,39.0,1.0,2.0,2.0,7.0,14.0,0.3925,6448.271112,7.8750
2,2.0,39.0,1.0,1.0,1.0,9.0,2.0,0.1433,12999.542264,9.1250
3,2.0,39.0,1.0,2.0,2.0,7.0,0.0,0.1783,12999.542264,7.0000
4,2.0,39.0,1.0,3.0,3.0,6.0,3.0,0.1833,12999.542264,6.1875
5,6.0,38.0,1.0,1.0,1.0,8.0,9.0,0.2700,8874.440799,8.5625
6,6.0,40.0,1.0,2.0,2.0,9.0,9.0,0.2883,8874.440799,9.5625
7,6.0,42.0,1.0,3.0,3.0,8.0,6.0,0.3016,8874.440799,8.3750
8,7.0,39.0,1.0,1.0,1.0,7.0,9.0,0.2808,6911.879921,7.5625
9,7.0,35.0,1.0,2.0,2.0,6.0,10.0,0.3233,6911.879921,6.6250
